In [ ]:
import polars as pl
import databpy as db
import numpy as np
from io import StringIO

# Updated JSON data
json_file = StringIO("""
{
  "Dino": [
    [55.3846, 97.1795, 0.0],
    [51.5385, 96.0256, 0.0]
  ],
  "Star": [
    [58.2136, 91.8819, 0.0],
    [58.1961, 92.215, 0.0]
  ],
  "Is_Visible": [
    [true],
    [false]
  ],
  "Intensity": [
    [10],
    [20]
  ],
  "My Strings": [
    ["A"],
    ["B"]
  ]
}
""")

# Load data into Polars DataFrame
df = pl.read_json(json_file)
columns_to_explode = [col for col in df.columns if df[col].dtype == pl.List(pl.List)]
df = df.explode(columns_to_explode)

# Create Bob object
vertices = np.zeros((len(df), 3), dtype=np.float32)
bob = db.create_bob(vertices, name="DinoStar")

# Store named attributes
for col in df.columns:
    
    # Skip columns that are string-typed
    if df[col].dtype == pl.List(pl.Utf8):
        print(f"Skipping column '{col}' as it contains string data.")
        continue

    data = df[col].to_numpy()
    if df[col].dtype == pl.List:
        data = np.vstack(data)
    bob.store_named_attribute(data, col)

# Check one of the named attributes
bob.named_attribute("Dino")

In [ ]:
import polars as pl
import databpy as db
from io import StringIO

json_file = StringIO("""
{
  "Dino": [
    [55.3846, 97.1795, 0.0],
    [51.5385, 96.0256, 0.0]
  ],
  "Star": [
    [58.2136, 91.8819, 0.0],
    [58.1961, 92.215, 0.0]
  ]
}
""")

df = pl.read_json(json_file)
columns_to_explode = [col for col in df.columns if df[col].dtype == pl.List(pl.List)]
df = df.explode(columns_to_explode)

vertices = np.zeros((len(df), 3), dtype=np.float32)
bob = db.create_bob(vertices, name="DinoStar")

for col in df.columns:
    data = np.vstack(df.get_column(col).to_numpy())
    bob.store_named_attribute(data, col)

bob.named_attribute("Dino")